In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [2]:
# 解析高德地图API网址信息
def get_latlng_gaode(address,ak):
    ak= ak
    url="http://restapi.amap.com/v3/geocode/geo?key=%s&address=%s"%(ak,address)
    data=requests.get(url)
    contest=data.json()
    return contest

In [3]:
# 获取高德地图API地址信息
def get_receiver_lnglat_gaode(address):
    start_time = time.time()
    re_List_gaode = []
    for b in address:
#         print(b)
        re_Dict_gaode = {}
        try:
            temp=get_latlng_gaode(b,ak_gaode)
        except requests.exceptions.ConnectionError: # HTTPError
            re_Dict_gaode["re_location_gaode"] = "请求出错"
            
        else:
            if ('geocodes' in temp):
                re_Dict_gaode["re_location_gaode"] = temp['geocodes'][0]['location']
                re_Dict_gaode["re_formatted_address_gaode"] = temp['geocodes'][0]['formatted_address']
                re_Dict_gaode["re_province_gaode"] = temp['geocodes'][0]['province']
                re_Dict_gaode["re_country_gaode"] = temp['geocodes'][0]['country']
                if ('citycode' in temp):
                    re_Dict_gaode["re_citycode_gaode"] = temp['geocodes'][0]['citycode']
                else:
                    pass
                re_Dict_gaode["re_city_gaode"] = temp['geocodes'][0]['city']
                re_Dict_gaode["re_district_gaode"] = temp['geocodes'][0]['district']
                re_Dict_gaode["re_level_gaode"] = temp['geocodes'][0]['level']
                re_Dict_gaode["re_township_gaode"] = temp['geocodes'][0]['township']
                re_Dict_gaode["re_adcode_gaode"] = temp['geocodes'][0]['adcode']
                re_Dict_gaode["re_street_gaode"] = temp['geocodes'][0]['street']
                re_Dict_gaode["re_number_gaode"] = temp['geocodes'][0]['number']
            else:
                pass
        re_List_gaode.append(re_Dict_gaode)
    #     time.sleep(1)
    end_time = time.time()
    print("高德总用时：",end_time-start_time)
    return re_List_gaode

In [4]:
# 解析百度地图API网址信息
def get_latlng_baidu(address,ak):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'#输出数据的格式
    ak = ak
    add = quote(address) #由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak
    req = urlopen(uri)
    res = req.read().decode()
    temp = json.loads(res)
    return temp

In [5]:
# 获取百度地图API地址信息
def get_receiver_lnglat_baidu(address):
    start_time = time.time()
    re_List_baidu = []
    for b in address:
#         print(b)
        re_Dict_baidu = {}
        try:
            temp = get_latlng_baidu(b,ak_baidu)
        except HTTPError as e:
            print("请求出错")
            pass
        else:
            if ('result' in temp):
                re_Dict_baidu["re_Lng_baidu"] = temp['result']['location']['lng']
                re_Dict_baidu["re_Lat_baidu"] = temp['result']['location']['lat']
                re_Dict_baidu["re_Precise_baidu"] = temp['result']['precise']
                re_Dict_baidu["re_Confidence_baidu"] = temp['result']['confidence']
                re_Dict_baidu["re_Comprehension_baidu"] = temp['result']['comprehension']
                re_Dict_baidu["re_Level_baidu"] = temp['result']['level']
            else:
                pass
        re_List_baidu.append(re_Dict_baidu)
    #     time.sleep(1)
    end_time = time.time()
    print("百度总用时：", end_time - start_time)
    return re_List_baidu

In [6]:
# ak码
ak_gaode = 'c71d9eda293d20db64955275557d92d4'
ak_baidu = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'

In [7]:
# 导入数据
inputpath = "G:\\work\\basic\\province\\DEC18\\"
inputFileName = "四川(所在省).xlsx"
data = pd.read_excel(inputpath + inputFileName)


In [8]:
data.columns 

Index(['Unnamed: 0', '订单编号', '订单状态', '订单子状态', '订单挂起原因', '安装失败原因', '订单来源', '店铺',
       '订单金额', '订单生成时间', '交易完成时间', '发货时间', '物流单回填时间', '入网用户姓名', '入网身份证号',
       '入网手机号', '收货人姓名', '收货地址', '收货人电话号码', '邮编', '所在省 / 市 / 县', '号码归属地',
       'CPS推荐人', '订单备注', '物流单号', '物流签收时间', '承运商', '大小卡类型', '写卡渠道', '销售品编号',
       '销售品名称', '销售品类型', '套餐', '主号码', '副号码', '现金预存款', '靓号低消', '靓号等级', '靓号预存款',
       '营业厅送货方式', '是否线下模式', '是否线下转线上', '线下转线上原因', '营业厅送货iccid', '所在省', '所在市',
       '所在县', '号码归属省', '号码归属市'],
      dtype='object')

In [9]:
data["承运商"].value_counts()

京东物流       20390
京东线下        5223
韵达          3096
EMS           86
骏飞通信专营店        5
顺丰             2
Name: 承运商, dtype: int64

### 选取有效字段

In [10]:
df = data[["所在省","所在市","收货地址","订单编号","承运商"]]

In [11]:
df["address_all"]= df["所在省"] +"省"+ df["所在市"] + df["收货地址"]

d:\python\envs\jupyterpy36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


### 文件分组并检查

In [12]:
print("data:" + str(data.shape))
print("df:" + str(df.shape))

data:(36567, 49)
df:(36567, 6)


In [13]:
df_EMS = df[df["承运商"] == "EMS"].reset_index(drop=True)
df_JDWL = df[df["承运商"] == "京东物流"].reset_index(drop=True)
df_YD = df[df["承运商"] == "韵达"].reset_index(drop=True)
df_JDXX = df[df["承运商"] == "京东线下"].reset_index(drop=True)

In [14]:
print("df_EMS:" + str(df_EMS.shape))
print("df_JDWL:" + str(df_JDWL.shape))
print("df_YD:" + str(df_YD.shape))
print("df_JDXX:" + str(df_JDXX.shape))

df_EMS:(86, 6)
df_JDWL:(20390, 6)
df_YD:(3096, 6)
df_JDXX:(5223, 6)


In [15]:
list_JDXX_gaode = get_receiver_lnglat_gaode(df_JDXX["address_all"])

高德总用时： 437.93904876708984


In [16]:
list_JDWL_gaode = get_receiver_lnglat_gaode(df_JDWL["address_all"])

高德总用时： 1655.1326682567596


In [17]:
list_EMS_gaode = get_receiver_lnglat_gaode(df_EMS["address_all"])

高德总用时： 6.291359901428223


In [18]:
list_YD_gaode = get_receiver_lnglat_gaode(df_YD["address_all"])

高德总用时： 283.89623761177063


In [19]:
# df化(高德)
df_EMS_gaode = pd.DataFrame(list_EMS_gaode)
df_JDXX_gaode = pd.DataFrame(list_JDXX_gaode)
df_JDWL_gaode = pd.DataFrame(list_JDWL_gaode)
df_YD_gaode = pd.DataFrame(list_YD_gaode)

data_JDWL_gaode = pd.concat([df_JDWL,df_JDWL_gaode],axis=1)
data_JDXX_gaode = pd.concat([df_JDXX,df_JDXX_gaode],axis=1)
data_EMS_gaode = pd.concat([df_EMS,df_EMS_gaode],axis=1)
data_YD_gaode = pd.concat([df_YD,df_YD_gaode],axis=1)

In [20]:
list_JDXX_baidu = get_receiver_lnglat_baidu(df_JDXX["address_all"])

百度总用时： 381.86984181404114


In [21]:
list_JDWL_baidu = get_receiver_lnglat_baidu(df_JDWL["address_all"])

百度总用时： 1653.1905570030212


In [22]:
list_EMS_baidu = get_receiver_lnglat_baidu(df_EMS["address_all"])

百度总用时： 4.435253858566284


In [23]:
list_YD_baidu = get_receiver_lnglat_baidu(df_YD["address_all"])

百度总用时： 181.3113706111908


In [24]:
# df化(百度)
df_EMS_baidu = pd.DataFrame(list_EMS_baidu)
df_JDXX_baidu = pd.DataFrame(list_JDXX_baidu)
df_JDWL_baidu = pd.DataFrame(list_JDWL_baidu)
df_YD_baidu = pd.DataFrame(list_YD_baidu)

data_JDWL_baidu = pd.concat([df_JDWL,df_JDWL_baidu],axis=1)
data_JDXX_baidu = pd.concat([df_JDXX,df_JDXX_baidu],axis=1)
data_EMS_baidu = pd.concat([df_EMS,df_EMS_baidu],axis=1)
data_YD_baidu = pd.concat([df_YD,df_YD_baidu],axis=1)

In [25]:
# 检查各df
print("data_JDWL_gaode:" + str(data_JDWL_gaode.shape))
print("data_JDXX_gaode:" + str(data_JDXX_gaode.shape))
print("data_YD_gaode:" + str(data_YD_gaode.shape))
print("data_EMS_gaode:" + str(data_EMS_gaode.shape))
print("*"*50)
print("data_JDWL_baidu:" + str(data_JDWL_baidu.shape))
print("data_JDXX_baidu:" + str(data_JDXX_baidu.shape))
print("data_YD_baidu:" + str(data_YD_baidu.shape))
print("data_EMS_baidu:" + str(data_EMS_baidu.shape))

data_JDWL_gaode:(20390, 17)
data_JDXX_gaode:(5223, 17)
data_YD_gaode:(3096, 17)
data_EMS_gaode:(86, 17)
**************************************************
data_JDWL_baidu:(20390, 12)
data_JDXX_baidu:(5223, 12)
data_YD_baidu:(3096, 12)
data_EMS_baidu:(86, 12)


In [26]:
data_SC_gaode_all = pd.concat([data_JDWL_gaode,data_JDXX_gaode,data_EMS_gaode,data_YD_gaode],axis=0)

In [27]:
data_SC_baidu_all = pd.concat([data_JDWL_baidu,data_JDXX_baidu,data_EMS_baidu,data_YD_baidu],axis=0)

In [28]:
# 检查合并后结构
print("data:" + str(data.shape))
print("df:" + str(df.shape))
print("data_SC_gaode_all:" + str(data_SC_gaode_all.shape))
print("data_SC_baidu_all:" + str(data_SC_baidu_all.shape))

data:(36567, 49)
df:(36567, 6)
data_SC_gaode_all:(28795, 17)
data_SC_baidu_all:(28795, 12)


In [29]:
data_SC_gaode_all.to_excel(r"G:\work\basic\province\1812Info_sichuan_gaode.xlsx")

In [30]:
data_SC_baidu_all.to_excel(r"G:\work\basic\province\1812Info_sichuan_baidu.xlsx")

In [ ]:
# 导出数据
outpath = "D:\\work\\basic\\province\\"
name_ANSI = "1812_info_alter_lnglat1.xlsx"
# name_UTF8 = "1812_info_alter_lnglat.csv"

In [ ]:
print("验证完整收货地址：")
print(add[0:3])
print("&%"*50)